<a href="https://colab.research.google.com/github/ShravankumarMR/BigData-Spark/blob/main/JsonAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyspark

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Restaurant Analysis").getOrCreate()


In [3]:
import os
#importing os to set environment variable
def install_java():
  !apt-get install -y openjdk-17-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"
!java -version
install_java()

openjdk version "17.0.16" 2025-07-15
OpenJDK Runtime Environment (build 17.0.16+8-Ubuntu-0ubuntu122.04.1)
OpenJDK 64-Bit Server VM (build 17.0.16+8-Ubuntu-0ubuntu122.04.1, mixed mode, sharing)


In [4]:
!apt-get update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 https://cli.github.com/packages stable InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [83.6 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,201 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,842 kB]
Get:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy

In [8]:
#load data into df
restaurants = spark.read.json("/content/Input/restaurants.json")
print("Total records loaded:" , restaurants.count())

print ("Sample records: \n\n", restaurants.head(5))

Total records loaded: 3772
Sample records: 

 [Row(address=Row(building='1007', coord=[-73.856077, 40.848447], street='Morris Park Ave', zipcode='10462'), borough='Bronx', cuisine='Bakery', grades=[Row(date=Row($date=1393804800000), grade='A', score=2), Row(date=Row($date=1378857600000), grade='A', score=6), Row(date=Row($date=1358985600000), grade='A', score=10), Row(date=Row($date=1322006400000), grade='A', score=9), Row(date=Row($date=1299715200000), grade='B', score=14)], name='Morris Park Bake Shop', restaurant_id='30075445'), Row(address=Row(building='469', coord=[-73.961704, 40.662942], street='Flatbush Avenue', zipcode='11225'), borough='Brooklyn', cuisine='Hamburgers', grades=[Row(date=Row($date=1419897600000), grade='A', score=8), Row(date=Row($date=1404172800000), grade='B', score=23), Row(date=Row($date=1367280000000), grade='A', score=12), Row(date=Row($date=1336435200000), grade='A', score=12)], name="Wendy'S", restaurant_id='30112340'), Row(address=Row(building='351', co

In [9]:
restaurants.printSchema()

root
 |-- address: struct (nullable = true)
 |    |-- building: string (nullable = true)
 |    |-- coord: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- street: string (nullable = true)
 |    |-- zipcode: string (nullable = true)
 |-- borough: string (nullable = true)
 |-- cuisine: string (nullable = true)
 |-- grades: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- date: struct (nullable = true)
 |    |    |    |-- $date: long (nullable = true)
 |    |    |-- grade: string (nullable = true)
 |    |    |-- score: long (nullable = true)
 |-- name: string (nullable = true)
 |-- restaurant_id: string (nullable = true)



In [10]:
restaurants.show(1,truncate=False)

+-------------------------------------------------------+-------+-------+-------------------------------------------------------------------------------------------------------------------------------+---------------------+-------------+
|address                                                |borough|cuisine|grades                                                                                                                         |name                 |restaurant_id|
+-------------------------------------------------------+-------+-------+-------------------------------------------------------------------------------------------------------------------------------+---------------------+-------------+
|{1007, [-73.856077, 40.848447], Morris Park Ave, 10462}|Bronx  |Bakery |[{{1393804800000}, A, 2}, {{1378857600000}, A, 6}, {{1358985600000}, A, 10}, {{1322006400000}, A, 9}, {{1299715200000}, B, 14}]|Morris Park Bake Shop|30075445     |
+-----------------------------------------------

In [11]:
restaurants.createOrReplaceTempView("restaurants")

In [17]:
q1 = spark.sql("select borough, count(*) as restaurants from restaurants group by borough")
print("Total restaurants per Borough: \n")
q1.show()

Total restaurants per Borough: 

+-------------+-----------+
|      borough|restaurants|
+-------------+-----------+
|       Queens|        738|
|     Brooklyn|        684|
|Staten Island|        158|
|    Manhattan|       1883|
|        Bronx|        309|
+-------------+-----------+



In [18]:
from pyspark.sql.functions import explode
df_arrayData = restaurants.select('restaurant_id', 'name', explode(restaurants.grades).alias('Grades'))
df_arrayData.show(10,truncate=False)

+-------------+------------------------------+------------------------+
|restaurant_id|name                          |Grades                  |
+-------------+------------------------------+------------------------+
|30075445     |Morris Park Bake Shop         |{{1393804800000}, A, 2} |
|30075445     |Morris Park Bake Shop         |{{1378857600000}, A, 6} |
|30075445     |Morris Park Bake Shop         |{{1358985600000}, A, 10}|
|30075445     |Morris Park Bake Shop         |{{1322006400000}, A, 9} |
|30075445     |Morris Park Bake Shop         |{{1299715200000}, B, 14}|
|30112340     |Wendy'S                       |{{1419897600000}, A, 8} |
|30112340     |Wendy'S                       |{{1404172800000}, B, 23}|
|30112340     |Wendy'S                       |{{1367280000000}, A, 12}|
|30112340     |Wendy'S                       |{{1336435200000}, A, 12}|
|30191841     |Dj Reynolds Pub And Restaurant|{{1409961600000}, A, 2} |
+-------------+------------------------------+------------------

In [19]:
df_arrayData.printSchema()

root
 |-- restaurant_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- Grades: struct (nullable = true)
 |    |-- date: struct (nullable = true)
 |    |    |-- $date: long (nullable = true)
 |    |-- grade: string (nullable = true)
 |    |-- score: long (nullable = true)



In [20]:
df2 = df_arrayData.select('restaurant_id', 'name', 'Grades.date', 'Grades.grade', 'Grades.score')
df2.show(10, truncate=False)

+-------------+------------------------------+---------------+-----+-----+
|restaurant_id|name                          |date           |grade|score|
+-------------+------------------------------+---------------+-----+-----+
|30075445     |Morris Park Bake Shop         |{1393804800000}|A    |2    |
|30075445     |Morris Park Bake Shop         |{1378857600000}|A    |6    |
|30075445     |Morris Park Bake Shop         |{1358985600000}|A    |10   |
|30075445     |Morris Park Bake Shop         |{1322006400000}|A    |9    |
|30075445     |Morris Park Bake Shop         |{1299715200000}|B    |14   |
|30112340     |Wendy'S                       |{1419897600000}|A    |8    |
|30112340     |Wendy'S                       |{1404172800000}|B    |23   |
|30112340     |Wendy'S                       |{1367280000000}|A    |12   |
|30112340     |Wendy'S                       |{1336435200000}|A    |12   |
|30191841     |Dj Reynolds Pub And Restaurant|{1409961600000}|A    |2    |
+-------------+----------

In [21]:
df2.registerTempTable("grade")
spark.sql("select restaurant_id, name, max(score) from grade group by restaurant_id, name").show()


/usr/local/lib/python3.12/dist-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


+-------------+--------------------+----------+
|restaurant_id|                name|max(score)|
+-------------+--------------------+----------+
|     40363151|             Olive'S|        13|
|     40363426|         Lexler Deli|        12|
|     40365644|Joe Allen Restaurant|        13|
|     40370519|         Divers Cove|        13|
|     40373889|  Felidia Restaurant|        13|
|     40375972|         Vegas Diner|        13|
|     40390814|        Loui Seafood|        13|
|     40391531|    Tea And Sympathy|        10|
|     40402145|Bali Nusa Indones...|        25|
|     40604740|     T.G.I. Friday'S|        18|
|     40649463|La Boina Roja Ste...|        15|
|     40698823|Union Cafe Restau...|        12|
|     40852397|         Candies Bar|        25|
|     40879245|Grand Hyatt New Y...|        40|
|     40369017|            Palm Too|        12|
|     40383799|        Jackson Hole|        14|
|     40394957|       Tuscany Grill|        19|
|     40397723|        White Castle|    

In [22]:
df4 = df2.select("restaurant_id", "name", "date.$date", "grade", "score")
df4.show()

+-------------+--------------------+-------------+-----+-----+
|restaurant_id|                name|        $date|grade|score|
+-------------+--------------------+-------------+-----+-----+
|     30075445|Morris Park Bake ...|1393804800000|    A|    2|
|     30075445|Morris Park Bake ...|1378857600000|    A|    6|
|     30075445|Morris Park Bake ...|1358985600000|    A|   10|
|     30075445|Morris Park Bake ...|1322006400000|    A|    9|
|     30075445|Morris Park Bake ...|1299715200000|    B|   14|
|     30112340|             Wendy'S|1419897600000|    A|    8|
|     30112340|             Wendy'S|1404172800000|    B|   23|
|     30112340|             Wendy'S|1367280000000|    A|   12|
|     30112340|             Wendy'S|1336435200000|    A|   12|
|     30191841|Dj Reynolds Pub A...|1409961600000|    A|    2|
|     30191841|Dj Reynolds Pub A...|1374451200000|    A|   11|
|     30191841|Dj Reynolds Pub A...|1343692800000|    A|   12|
|     30191841|Dj Reynolds Pub A...|1325116800000|    A

In [28]:
from pyspark.sql.functions import from_unixtime
from pyspark.sql.functions import col

df5=df4.select("restaurant_id", "name", from_unixtime(col("$date")/1000, "yyyy-MM-dd").alias("grade_dt"))
df5.show()

+-------------+--------------------+----------+
|restaurant_id|                name|  grade_dt|
+-------------+--------------------+----------+
|     30075445|Morris Park Bake ...|2014-03-03|
|     30075445|Morris Park Bake ...|2013-09-11|
|     30075445|Morris Park Bake ...|2013-01-24|
|     30075445|Morris Park Bake ...|2011-11-23|
|     30075445|Morris Park Bake ...|2011-03-10|
|     30112340|             Wendy'S|2014-12-30|
|     30112340|             Wendy'S|2014-07-01|
|     30112340|             Wendy'S|2013-04-30|
|     30112340|             Wendy'S|2012-05-08|
|     30191841|Dj Reynolds Pub A...|2014-09-06|
|     30191841|Dj Reynolds Pub A...|2013-07-22|
|     30191841|Dj Reynolds Pub A...|2012-07-31|
|     30191841|Dj Reynolds Pub A...|2011-12-29|
|     40356018|     Riviera Caterer|2014-06-10|
|     40356018|     Riviera Caterer|2013-06-05|
|     40356018|     Riviera Caterer|2012-04-13|
|     40356018|     Riviera Caterer|2011-10-12|
|     40356068|  Tov Kosher Kitchen|2014

In [29]:
df5.printSchema()

root
 |-- restaurant_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- grade_dt: string (nullable = true)



In [30]:
from pyspark.sql.functions import to_date
df6 = df5.withColumn("grade_dt", to_date( col("grade_dt"), "yyyy-MM-dd" ))
df6.show()

+-------------+--------------------+----------+
|restaurant_id|                name|  grade_dt|
+-------------+--------------------+----------+
|     30075445|Morris Park Bake ...|2014-03-03|
|     30075445|Morris Park Bake ...|2013-09-11|
|     30075445|Morris Park Bake ...|2013-01-24|
|     30075445|Morris Park Bake ...|2011-11-23|
|     30075445|Morris Park Bake ...|2011-03-10|
|     30112340|             Wendy'S|2014-12-30|
|     30112340|             Wendy'S|2014-07-01|
|     30112340|             Wendy'S|2013-04-30|
|     30112340|             Wendy'S|2012-05-08|
|     30191841|Dj Reynolds Pub A...|2014-09-06|
|     30191841|Dj Reynolds Pub A...|2013-07-22|
|     30191841|Dj Reynolds Pub A...|2012-07-31|
|     30191841|Dj Reynolds Pub A...|2011-12-29|
|     40356018|     Riviera Caterer|2014-06-10|
|     40356018|     Riviera Caterer|2013-06-05|
|     40356018|     Riviera Caterer|2012-04-13|
|     40356018|     Riviera Caterer|2011-10-12|
|     40356068|  Tov Kosher Kitchen|2014

In [31]:
df6.createOrReplaceTempView("dateWiseTable")
spark.sql("select name, year(grade_dt), count(*) from dateWiseTable where name = 'Morris Park Bake Shop' GROUP BY name,year(grade_dt)").show()

+--------------------+--------------+--------+
|                name|year(grade_dt)|count(1)|
+--------------------+--------------+--------+
|Morris Park Bake ...|          2014|       1|
|Morris Park Bake ...|          2011|       2|
|Morris Park Bake ...|          2013|       2|
+--------------------+--------------+--------+



In [32]:
df_arrayCoord = restaurants.select('restaurant_id', 'name', restaurants.address.coord[0].alias('longitude'),restaurants.address.coord[1].alias('latitude'))
df_arrayCoord.show()


+-------------+--------------------+------------------+-----------------+
|restaurant_id|                name|         longitude|         latitude|
+-------------+--------------------+------------------+-----------------+
|     30075445|Morris Park Bake ...|        -73.856077|        40.848447|
|     30112340|             Wendy'S|        -73.961704|        40.662942|
|     30191841|Dj Reynolds Pub A...|-73.98513559999999|       40.7676919|
|     40356018|     Riviera Caterer|-73.98241999999999|        40.579505|
|     40356068|  Tov Kosher Kitchen|       -73.8601152|       40.7311739|
|     40356151|Brunos On The Bou...|       -73.8803827|       40.7643124|
|     40356442|       Kosher Island|       -74.1377286|       40.6119572|
|     40356483|  Wilken'S Fine Food|       -73.9068506|       40.6199034|
|     40356649|     Regina Caterers|-74.00528899999999|        40.628886|
|     40356731|Taste The Tropics...|       -73.9482609|       40.6408271|
|     40357217|           Wild Asia|  